In [1]:
cd ..

/home/tommy/Scrivania/Polimi/Honours Programme/SCFR


In [2]:
from data_structures.cfr_trees import *
from data_structures.trees import *

In [3]:
tree = randomTree(4,2,1)
cfr_tree = CFRTree(tree)

In [4]:
cfr_tree.buildJointFromMarginals()

(<data_structures.cfr_trees.CFRNode object at 0x7f3c6842cc18>, {2: 1, 14: 0}, 0.25)
(<data_structures.cfr_trees.CFRNode object at 0x7f3c6842c828>, {1: 1, 9: 0}, 0.25)
(<data_structures.cfr_trees.CFRNode object at 0x7f3c6842ca20>, {2: 0, 11: 1}, 0.25)
(<data_structures.cfr_trees.CFRNode object at 0x7f3c6842cc50>, {2: 1, 14: 1}, 0.25)
(<data_structures.cfr_trees.CFRNode object at 0x7f3c6842c860>, {1: 1, 9: 1}, 0.25)
(<data_structures.cfr_trees.CFRNode object at 0x7f3c6842ca90>, {2: 0, 12: 0}, 0.25)
(<data_structures.cfr_trees.CFRNode object at 0x7f3c6842c6a0>, {1: 0, 7: 0}, 0.25)
(<data_structures.cfr_trees.CFRNode object at 0x7f3c6842cac8>, {2: 0, 12: 1}, 0.25)
(<data_structures.cfr_trees.CFRNode object at 0x7f3c6842c6d8>, {1: 0, 7: 1}, 0.25)
(<data_structures.cfr_trees.CFRNode object at 0x7f3c6842c8d0>, {1: 1, 10: 0}, 0.25)
(<data_structures.cfr_trees.CFRNode object at 0x7f3c6842c908>, {1: 1, 10: 1}, 0.25)
(<data_structures.cfr_trees.CFRNode object at 0x7f3c6842c748>, {8: 0, 1: 0}, 0.2

[({1: 0, 2: 1, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0}, 0.25),
 ({1: 1, 2: 0, 7: 1, 8: 1, 9: 0, 10: 0, 11: 0, 12: 0, 13: 1, 14: 1}, 0.25),
 ({1: 0, 2: 0, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1}, 0.25),
 ({1: 1, 2: 1, 7: 0, 8: 0, 9: 1, 10: 1, 11: 0, 12: 0, 13: 1, 14: 1}, 0.0),
 ({1: 1, 2: 1, 7: 0, 8: 0, 9: 1, 10: 1, 11: 0, 12: 0, 13: 1, 14: 1}, 0.0),
 ({1: 1, 2: 1, 7: 0, 8: 0, 9: 1, 10: 1, 11: 0, 12: 0, 13: 1, 14: 1}, 0.0),
 ({1: 1, 2: 1, 7: 0, 8: 0, 9: 1, 10: 1, 11: 0, 12: 0, 13: 1, 14: 1}, 0.0),
 ({1: 1, 2: 1, 7: 0, 8: 0, 9: 1, 10: 1, 11: 0, 12: 0, 13: 1, 14: 1}, 0.0),
 ({1: 1, 2: 1, 7: 0, 8: 0, 9: 1, 10: 1, 11: 0, 12: 0, 13: 1, 14: 1}, 0.0),
 ({1: 1, 2: 1, 7: 0, 8: 0, 9: 1, 10: 1, 11: 0, 12: 0, 13: 1, 14: 1}, 0.0)]

In [5]:
cfr_tree.root.buildRealizationForm(0, 1)

In [6]:
leaves = set()
cfr_tree.root.find_terminals(leaves)

In [7]:
(plan, w) = cfr_tree.builSupportingPlan(leaves.pop(), 0)

In [8]:
cfr_tree.root.terminalsUnderPlan(0, plan)

In [9]:
for l in leaves:
    print(l.omega)

0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25


In [10]:
tree.display()

Player 1 - Infoset 0 - Node 0
Player 0 - Infoset 1 - Node 1 (children of Node0 via Action 0.0)
Player 0 - Infoset 2 - Node 2 (children of Node0 via Action 0.1)
Player 1 - Infoset 3 - Node 3 (children of Node1 via Action 1.0)
Player 1 - Infoset 4 - Node 4 (children of Node1 via Action 1.1)
Player 0 - Infoset 7 - Node 7 (children of Node3 via Action 3.0)
Player 0 - Infoset 8 - Node 8 (children of Node3 via Action 3.1)
Leaf15 (children of Node7 via Action 7.0) -  utility is [32, 12]
Leaf16 (children of Node7 via Action 7.1) -  utility is [12, 61]
Leaf17 (children of Node8 via Action 8.0) -  utility is [73, 11]
Leaf18 (children of Node8 via Action 8.1) -  utility is [34, 74]
Player 0 - Infoset 9 - Node 9 (children of Node4 via Action 4.0)
Player 0 - Infoset 10 - Node 10 (children of Node4 via Action 4.1)
Leaf19 (children of Node9 via Action 9.0) -  utility is [46, 90]
Leaf20 (children of Node9 via Action 9.1) -  utility is [12, 3]
Leaf21 (children of Node10 via Action 10.0) -  utility is [

In [11]:
tree.root.children[0].getSequence(1)

{0: 0}

In [12]:
cfr_tree.information_sets[1].children_infoset

[[<InfoSet8 - Player0>, <InfoSet7 - Player0>],
 [<InfoSet9 - Player0>, <InfoSet10 - Player0>]]

In [13]:
cfr_tree.information_sets[14].children_leaves

[[<data_structures.cfr_trees.CFRNode at 0x7f3c6842cc18>],